### Mitchell

**Station Name:** 	mchl (mchl00aus)

**Location:** Walhallow, Queensland, Austalia

**Ellipsoidal Coordinates:**

- Latitude: -26.359 degrees

- Longitude: 148.145 degrees

- Height: 534.591 meters

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/MCHL.sta)

[Google Maps Link](https://www.google.com/maps/place/26%C2%B021'32.4%22S+148%C2%B008'42.0%22E/@-26.359,148.145,11z/data=!4m5!3m4!1s0x0:0x9200f9ebb23ec5b1!8m2!3d-26.359!4d148.145?hl=en) 

<p align=center>
<img src="https://www.unavco.org/data/gps-gnss/lib/images/station_images/MCHL.jpg" width="500">
</p>

**Archives:** CDDIS, BKG, GA
 
Read the instructions for the [soil moisture code!](https://gnssrefl.readthedocs.io/en/latest/pages/README_vwc.html)!

In [ ]:
from gnssrefl.vwc import vwc
from gnssrefl.quickPhase import quickphase
from gnssrefl.vwc_input import vwc_input
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions

#### Step 1: GNSS-IR
Begin by generating the SNR files. To be sure we can get the L2C data, we will use the RINEX 3 files.
These require the longer station name (mchl00aus) and are available at either cddis or ga.
Choose the one that is less slow for you. 
In some cases we hae found the data are at cddis but not ga or vice versa. Apparently the data are also 
available from BKG. We are not sure why this is. If you have a way to access 
these files, just download them and put them in the local
processing area and use the -nolook option). We are going to start with two years of data and 
using the default GPS orbit option and the cddis archive:

In [ ]:
rinex2snr("mchl00aus", 2017, 1, doy_end=365, year_end=2018, archive="cddis")

If you have any difficult downloading these RINEX 3 files, UNAVCO has kindly downloaded and converted 
a few years of files into RINEX 2.11 that includes L2C data. Uncomment the line below if you want to use this:

In [ ]:
# rinex2snr('mchl', 2017, 1, doy_end=365, archive='special')

Use the <code>quickLook</code> with the l2c frequency to give a look to the data quality.
Then set up your parameters with <code>make_gnssir_input</code>

In [ ]:
make_gnssir_input("mchl", l2c=True)

The location of the json file is printed to the screen. 
The default behavior is to accept all azimuths.
Modify the azimuths in the json if you feel that is needed.
How can you tell if you have bad azimuths? Look back at the <code>quickLook</code> results.
Do not worry excessively about this at this stage. Once you have a VWC solution you can go back and easily/quickly iterate to remove bad 
azimuths and satellite tracks.

Run the <code>gnssir</code> each day in 2017 and 2018:

In [ ]:
gnssir("mchl", 2017, 1, doy_end=365, year_end=2018)

#### Step 2: Soil Moisture

Pick the satellite tracks you are going to use:


In [ ]:
vwc_input?

In [ ]:
vwc_input("mchl", 2017)

This creates a file that will go in $REFL_CODE/input/mchl_phaseRH.txt

This file can be hand edited if you find out later that a 
particularly satellite track is not working. To comment lines out you use %.

Run the <code>phase</code> code:

In [ ]:
quickphase("mchl", 2017, 1, doy_end=365, year_end=2018)

The location of the output files for this stage are printed to the screen.

Finally, the <code>vwc</code> module compiles all the 
data in the requested years and generates a volumetric water content file.

It also produces plots which can help you assess your results:

1. Raw phases in geographic quadrants
2. Daily phase averages
3. Modeling Results
4. Final VWC

In [ ]:
vwc("mchl", 2017, year_end=2018)

A big thank you to Lucas Holden for test driving this use case.